# Segmenting and Clustering Toronto(canada) Neighborhoods 

### Import the necessary libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import csv

import pandas as pd
import numpy as np

## Get the html from the wikipedia page with the postal codes of Canada

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

Now that we have the html, we need to find the table in it.

In [3]:
table = soup.find('table')

Also, we need to create the csv file in which we will save the necessary data

In [4]:
csv_file = open('csv_toronto_neighbourhood.csv', 'w')
csv_writer = csv.writer(csv_file)

## Write the header in the csv file

In [5]:
row = []

for th in table.find_all('th'):
    row_value = th.text.rstrip()
    
    row.append(row_value)
print(row)
csv_writer.writerow(row)

['Postcode', 'Borough', 'Neighbourhood']


32

## Write all the data on the csv file

It's important to remove the data with no Borough assigned and write the Borough name if no neighbourhood is assigned

In [6]:
for tr in table.find_all('tr'):
    i=0
    row=[]
    discard = False
    postalCode = None
    borough = None
    neighbourhood = None
    for td in tr.find_all('td'):
        if i==0:
            postalCode = td.text.rstrip()
        elif i==1:
            if (td.text.rstrip() == 'Not assigned'):
                discard = True
            else:
                borough = td.text.rstrip()
        else:
            i=0
            if (td.text.rstrip() == 'Not assigned'):
                neighbourhood = borough
            else:
                neighbourhood = td.text.rstrip()
        i=i+1
    if (discard == False):
        row.append(postalCode)
        row.append(borough)
        row.append(neighbourhood)
        csv_writer.writerow(row)
        #print(row)
        
        

In [7]:
csv_file.close()

## Open the new csv file

In [8]:
df = pd.read_csv('csv_toronto_neighbourhood.csv')
df.head()

,Postcode,Borough,Neighbourhood
0,NaN,NaN,NaN
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park


In [9]:
df.shape

(212, 3)

## Group Neighbourhoods with the same Postcode

In [10]:
df_mod = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
df_mod = df_mod.reset_index()
df_mod.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df_mod.shape

(103, 3)

## Get the latitue and longitude of each Postal code

If the geocoder is working, this code will get the latitude and longitude for all the Postal codes in Toronto

In [12]:
#import geocoder
#i=0
#latitude = []
#longitude = []
#for i in range(len(df_mod)):
#    lat_lng_coords = None
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(df_mod.iloc[i]['Postcode']))
#        lat_lng_coords = g.latlng
#
#    latitude.append(lat_lng_coords[0])
#    longitude.append(lat_lng_coords[1])
#    
#print(len(latitute), len(longitude))

If geocoder is not working, we can download the CSV from http://cocl.us/Geospatial_data

In [13]:
import requests

url = 'http://cocl.us/Geospatial_data'
df_geo=pd.read_csv(url)

In [14]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df_geo.shape

(103, 3)

We can see that the Postal code DataFrame and the Dataframe we created have the sabe ammount of row. Now we should join both of them in one single dataframe.

In [16]:
df_toronto = df_mod.join(df_geo.set_index('Postal Code'), on='Postcode')

In [19]:
df_toronto.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
